In [25]:
import pandas as pd
import numpy as np
import json

In [26]:
# 原始文件读取
df = pd.read_csv("D:/dblp_full.csv",usecols = ['id','author_id','references','year'])

In [34]:
# 获取作者、关键词、引文和文章id信息
authors = list(df['author_id'])
keywords = list(df['new_keywords'])
refs = list(df['references'])
ids = list(df['id'].astype(str))

# 统计作者、论文和关键词信息

In [40]:
id_df = list(df['id'].astype(str))
author_df = list(df['author_id'])
keyword_df = list(df['new_keywords'])
aus = []
kws = []
print("不重复的论文数为{}".format(len(set(id_df))))
for au in author_df:
    if type(au)!=float and len(au)>0:
        aus += au.split(";")
print("不重复的作者数为{}".format(len(set(aus))))
for kw in keyword_df:
    if type(kw)!=float and len(kw)>0:
        kws += kw.split(";")
print("不重复的关键词数为{}".format(len(set(kws))))

不重复的论文数为103424
不重复的作者数为137438
不重复的关键词数为248357


# 获取node_type文件 

In [41]:
for i in range(103424):
    p.append(str(i)+" "+"P")
for j in range(103424,240862):
    a.append(str(j)+" "+"A")
for m in range(240862,489219):
    k.append(str(m)+" "+"K")
total = p+a+k
with open("D:/dblp_node_type.txt","w") as f:
    for each in total:
        f.write(each+"\n")
    f.close()

In [ ]:
# 作者 关键词 论文ID化

In [1]:
def get_refs_ids(x,paper_ids):
    refs = []
    if type(x)!=float and len(x)>0:
        for ref in x.split(";"):
            refs.append(str(paper_ids.index(str(ref))))
    return ";".join(refs)

def get_paper_id(x,paper_ids):
    x = str(paper_ids.index(str(x)))
    return x
def get_author_id(x,author_ids):
    aus_x = []
    if type(x)!=float and len(x)>0:
        
        for au in x.split(";"):
            aus_x.append(str(author_ids.index(au)+103424))
    return ";".join(aus_x)
def get_keyword_id(x,keyword_ids):
    kws_x = []
    if type(x)!=float and len(x)>0:
        
        for kw in x.split(";"):
            kws_x.append(str(keyword_ids.index(kw)+240862))
    return ";".join(kws_x)
paper_ids = list(set(id_df))
author_ids = list(set(aus))
keyword_ids = list(set(kws))
df['paper_id'] = df['id'].apply(get_paper_id,paper_ids = paper_ids)
print("文章id化结束")
df['author_id'] = df['author_id'].apply(get_author_id,author_ids = author_ids)
print("作者id化结束")
df['refs_id'] = df['references'].apply(get_refs_ids,paper_ids = paper_ids)
print("引文id化结束")
df['keyword_id'] = df['new_keywords'].apply(get_keyword_id,keyword_ids = keyword_ids)
print("关键词id化结束")
df.to_csv("D:/dblp.csv",index=False)

# 候选文献数据集/目标文献数据集划分

In [47]:
orgin = df[df['year']>2011]
print(len(orgin))
orgin_ids = list(orgin['paper_id'].astype(str))
orgin_refs = list(orgin['refs_id'])
useful = []
for x,y in zip(orgin_ids,orgin_refs):
    if type(y)!=float and len(y)>0:
        useful.append(x)
train_df = df[df['paper_id'].isin(useful)]
paper_IDS = list(df['paper_id'].astype(str))
test = [each for each in paper_IDS if each not in useful]
test_df = df[df['paper_id'].isin(test)]
train_df.to_csv("D:/dblp_train.csv",index=False)
test_df.to_csv("D:/dblp_test.csv",index=False)

7295


7149

# 获取训练集/测试集的边关系：论文-关键字/作者-论文

In [55]:
#train.txt  2014年前的边关系 pd.concat(axis=0)
#valid.txt 2014年后的边关系，只包括文章与文章的边关系
#features.txt 包括所有节点的特征关系
#node_type.txt 节点边的类型
# 1：作者-撰写-论文
# 2：论文-包含-关键词
# 3：论文-引用-论文
# valid.txt增加三种关系的负例
# test.txt增加三种关系的负例
# 用阿拉伯数字表示1-16707表示为论文编号16708-35568表示为作者编号  35569-44057表示为关键词编号
def get_author_paper_edges(authors,paper):
    a_p_edges = []
    normal_a_p_edges = []
    for i,author in enumerate(authors):
        if type(author)!= float and len(author)>0:
            for au in author.split(";"):
                a_p_edges.append("1 "+ au + " " + paper[i])
                normal_a_p_edges.append(au + " " + paper[i])
    return a_p_edges, normal_a_p_edges
def get_paper_keyword_edges(papers,keywords):
    p_k_edges = []
    normal_p_k_edges = []
    for i,keyword in enumerate(keywords):
        if type(keyword)!= float and len(keyword)>0:
            for kw in keyword.split(";"):
                p_k_edges.append("2 " + papers[i] + " " + kw )
                normal_p_k_edges .append(papers[i] + " "+ kw)
    return p_k_edges, normal_p_k_edges
def get_paper_paper_edges(papers,refs):
    p_p_edges = []
    normal_p_p_edges = []
    for i,ref in enumerate(refs):
        if type(ref)!= float and len(ref)>0:
            for r in ref.split(";"):
                p_p_edges.append("3 " + papers[i] + " " + r )
                normal_p_p_edges .append(papers[i] + " "+ r)
    return p_p_edges, normal_p_p_edges     
def write_text(texts,filepath):
    with open(filepath,"w",encoding = "utf-8") as f:
        for each in texts:
            f.write(each+"\n")
        f.close()
train_paper = list(map(lambda x: str(x),list(train_df['paper_id'])))
train_author = list(train_df['author_id'])
train_keywords = list(train_df['keyword_id'])
train_refs =  list(train_df['refs_id'])
train_a_p_edges,normal_train_a_p_edges = get_author_paper_edges(train_author,train_paper)
train_p_k_edges,normal_p_k_edges =  get_paper_keyword_edges(train_paper,train_keywords)
train_p_p_edges,normal_train_p_p_edges = get_paper_paper_edges(train_paper,train_refs)
gatne_train_edges = train_a_p_edges + train_p_k_edges + train_p_p_edges
normal_train_edges = normal_train_a_p_edges + normal_p_k_edges + normal_train_p_p_edges
write_text(gatne_train_edges,"D:/datasets/dblp_gatne_train.txt")
write_text(normal_train_edges,"D:/datasets/dblp_normal_train.txt")
train_paper_ids = list(map(lambda x: str(x),list(train_df['paper_id'])))

# 目标文献数据集正确引文获取及目标文献数据集更新

In [63]:
import json
test_paper_ids = list(map(lambda x: str(x),list(test_df['paper_id'])))
test_refs = list(test_df['refs_id'])
true_cites={}
for x,y in zip(test_paper_ids,test_refs):
    new_true_cites = []
    if type(y)!=float and len(y)>0:
         for each in y.split(";"):
            if each in train_paper_ids:
                    new_true_cites.append(each)
         if len(new_true_cites)==0:
            continue
         else:
            true_cites[x] = new_true_cites
json.dump(true_cites,open("D:/datasets/dblp_true_cites.json","w")) 
last_true_ids = list(true_cites.keys())
last_test_df = test_df[test_df['paper_id'].astype(str).isin(last_true_ids)]
last_test_df.to_csv("D:/datasets/dblp_test.csv",index = False)